In [1]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

import sounding_utils
from importlib import reload

from functools import partial

import os
from os.path import join
import subprocess

import sys
sys.path.append('../../') # lets us import ptype package from the subdir
#import ptype.

from joblib import Parallel, delayed

from xhistogram.xarray import histogram

import time

In [2]:
import batch_jobs.xr_map_reduce as xmr

In [7]:
dirpaths = ['/glade/scratch/dkimpara/ptype_case_studies/kentucky/rap/20220224/0000',
            '/glade/scratch/dkimpara/ptype_case_studies/kentucky/rap/20220224/0600']
    
results = Parallel(n_jobs=-1)(delayed(xmr.xr_map)(path, xmr.compute_func) for path in dirpaths)

res = xr.concat(results, dim=('time')) #each result ds will be for a different time
res.to_netcdf(f'/glade/work/dkimpara/ptype-aggs/test.nc')


In [17]:
nulls = res.isnull().sum()
for var in nulls.keys():
    print(nulls[var].values)

0
0
0
0
0
0
0
0
0
0
0


In [18]:
res

<xarray.Dataset>
Dimensions:            (predtype: 8, case_study_day: 1, time: 2, step: 18,
                        heightAboveGround: 21, bin: 999)
Coordinates:
  * predtype           (predtype) object 'ML_cicep' 'cicep' ... 'crain'
  * case_study_day     (case_study_day) object 'kentucky-20220224'
  * time               (time) int64 1645660800 1645682400
  * step               (step) float64 1.0 2.0 3.0 4.0 ... 15.0 16.0 17.0 18.0
  * heightAboveGround  (heightAboveGround) int64 0 250 500 ... 4500 4750 5000
  * bin                (bin) float64 -59.95 -59.85 -59.75 ... 39.65 39.75 39.85
Data variables:
    num_obs            (predtype, case_study_day, step, time) int64 dask.array<chunksize=(1, 1, 1, 1), meta=np.ndarray>
    t_h_fabv           (predtype, case_study_day, step, time) float64 dask.array<chunksize=(1, 1, 1, 1), meta=np.ndarray>
    dpt_h_fabv         (predtype, case_study_day, step, time) float64 dask.array<chunksize=(1, 1, 1, 1), meta=np.ndarray>
    wb_h_fabv          (predtype, case_study_day, step, time) float64 dask.array<chunksize=(1, 1, 1, 1), meta=np.ndarray>
    t_h_mean           (predtype, case_study_day, step, time, heightAboveGround) float32 dask.array<chunksize=(1, 1, 1, 1, 21), meta=np.ndarray>
    dpt_h_mean         (predtype, case_study_day, step, time, heightAboveGround) float32 dask.array<chunksize=(1, 1, 1, 1, 21), meta=np.ndarray>
    wb_h_mean          (predtype, case_study_day, step, time, heightAboveGround) float32 dask.array<chunksize=(1, 1, 1, 1, 21), meta=np.ndarray>
    t_h_hist           (predtype, case_study_day, step, time, heightAboveGround, bin) float64 dask.array<chunksize=(1, 1, 1, 1, 21, 999), meta=np.ndarray>
    dpt_h_hist         (predtype, case_study_day, step, time, heightAboveGround, bin) float64 dask.array<chunksize=(1, 1, 1, 1, 21, 999), meta=np.ndarray>
    wb_h_hist          (predtype, case_study_day, step, time, heightAboveGround, bin) float64 dask.array<chunksize=(1, 1, 1, 1, 21, 999), meta=np.ndarray>
    valid_time         (case_study_day, time, step) float64 1.646e+09 ... 1.6...